# 과제
[AI Hub](https://www.aihub.or.kr/)에 공개되어 있는 한국어 데이터셋을 활용하여 한국어 워드 임베딩 구축
* 임베딩까지 하는 게 기본
* 되면 레이어를 만들어서 딥러닝을 돌려보는 것도 좋다
* 한국어를 임베딩시켜서 모델에 넣는 것을 미리 하는 것

In [23]:
import pandas as pd
import numpy as np
import json
import csv
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [7]:
# JSON 파일 경로
json_file_path = "/content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.json"

# JSON 파일을 DataFrame으로 읽기
df = pd.read_json(json_file_path)

# CSV 파일로 저장
csv_file_path = "/content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training.csv"
df.to_csv(csv_file_path, index=False)

print(f"CSV 파일이 성공적으로 생성되었습니다. 경로: {csv_file_path}")

CSV 파일이 성공적으로 생성되었습니다. 경로: /content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training.csv


In [8]:
# xlsx 파일 경로
xlsx_file_path = "/content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.xlsx"

# xlsx 파일을 DataFrame으로 읽기
df = pd.read_excel(xlsx_file_path)

# CSV 파일로 저장
csv_file_path = "/content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.csv"
df.to_csv(csv_file_path, index=False)

print(f"CSV 파일이 성공적으로 생성되었습니다. 경로: {csv_file_path}")

CSV 파일이 성공적으로 생성되었습니다. 경로: /content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.csv


In [9]:
df = pd.read_csv('/content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training.csv')
df

,profile,talk
0,"{'persona-id': 'Pro_05349', 'persona': {'perso...","{'id': {'profile-id': 'Pro_05349', 'talk-id': ..."
1,"{'persona-id': 'Pro_05349', 'persona': {'perso...","{'id': {'profile-id': 'Pro_05349', 'talk-id': ..."
2,"{'persona-id': 'Pro_05349', 'persona': {'perso...","{'id': {'profile-id': 'Pro_05349', 'talk-id': ..."
3,"{'persona-id': 'Pro_05349', 'persona': {'perso...","{'id': {'profile-id': 'Pro_05349', 'talk-id': ..."
4,"{'persona-id': 'Pro_05349', 'persona': {'perso...","{'id': {'profile-id': 'Pro_05349', 'talk-id': ..."
...,...,...
51623,"{'persona-id': 'Pro_10870', 'persona': {'perso...","{'id': {'profile-id': 'Pro_10870', 'talk-id': ..."
51624,"{'persona-id': 'Pro_12450', 'persona': {'perso...","{'id': {'profile-id': 'Pro_12450', 'talk-id': ..."
51625,"{'persona-id': 'Pro_12457', 'persona': {'perso...","{'id': {'profile-id': 'Pro_12457', 'talk-id': ..."
51626,"{'persona-id': 'Pro_11368', 'persona': {'perso...","{'id': {'profile-id': 'Pro_11368', 'talk-id': ..."


In [10]:
df2 = pd.read_csv('/content/drive/MyDrive/KDT 시즌2/과제/data/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.csv')
df2

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51625,51626,노년,남성,재정,만성질환 무,분노,성가신,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,51627,노년,여성,재정,만성질환 무,불안,초조한,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
51627,51628,노년,여성,재정,만성질환 무,상처,희생된,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
51628,51629,노년,여성,대인관계,만성질환 무,불안,걱정스러운,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  51630 non-null  int64 
 1   연령          51630 non-null  object
 2   성별          51630 non-null  object
 3   상황키워드       51630 non-null  object
 4   신체질환        51630 non-null  object
 5   감정_대분류      51630 non-null  object
 6   감정_소분류      51630 non-null  object
 7   사람문장1       51630 non-null  object
 8   시스템문장1      51630 non-null  object
 9   사람문장2       51630 non-null  object
 10  시스템문장2      51630 non-null  object
 11  사람문장3       42695 non-null  object
 12  시스템문장3      42695 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.1+ MB


In [13]:
df2.drop(['Unnamed: 0', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류'], axis=1, inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사람문장1   51630 non-null  object
 1   시스템문장1  51630 non-null  object
 2   사람문장2   51630 non-null  object
 3   시스템문장2  51630 non-null  object
 4   사람문장3   42695 non-null  object
 5   시스템문장3  42695 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB


In [14]:
df2.dropna()

,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
304,지금까지 힘들게 일했는데 은퇴해서 돈이 없다고 하니 자식이 화를 내서 상처를 받았어.,돈이 없다고 하니 자식이 화를 내서 상처를 받으셨군요.,너무 화가 나. 도와줬던 건 기억 못 하고 더 받을 생각만 하니.,자식들이 받을 생각만 해서 화가 나셨군요. 이 기분을 바꿀 만한 일이 있을까요?,앞으로는 자식보다 나를 생각하며 살아야겠어. 자식 위해 살아도 필요 없다는 말 이제...,이제는 자식보다 나를 생각하며 살려 하시는군요.
305,친구한테 은퇴할 거라고 얘기했더니 앞으로 뭘 먹고 살 거냐면서 비웃더라고. 기분이 ...,비웃는 친구의 말에 기분이 나쁘셨군요.,아주 나쁜 친구야. 은퇴를 내가 하고 싶어 하는 것도 아닌데 말이야.,은퇴를 비웃는 친구가 나쁘다고 생각하시는군요. 앞으로 어떻게 하실 생각이신가요?,은퇴 후 조금 쉬며 즐기려고 해. 내가 잘할 수 있는 일이 분명 있을 테니까.,은퇴 후에는 쉬면서 즐기려고 할 생각이시군요.
306,친구한테 은퇴한다고 했더니 그게 말이나 되는 거냐며 날 한심한 사람 취급해서 서운했어.,친구분이 은퇴를 한심하게 취급해서 서운하셨군요.,맞아. 그동안 내가 얼마나 힘들게 일했는지 알고 있으면서 그런 말을 하다니.,얼마나 힘들었는지 아는 친구분이 그런 말을 했군요. 기분이 나아질 만한 방법이 있을까요?,감정을 추스르고 나중에 연락을 해야겠어. 지금은 많이 서운해.,감정을 추스르고 나중에 연락을 할 생각이시군요.
307,그동안 열심히 달려와서 좀 쉬려고 하는데 은퇴한다고 하니 주변에서 다 말려서 기분이...,그동안 너무 고생하셨어요. 주변 분들이 많이 걱정해 주시나 봐요.,응. 그렇지만 내 결정에 다들 냉담하게 반응하니 기분이 좋지 않아.,다들 냉담하게 반응해 기분이 좋지 않으시군요. 이 기분을 어떻게 바꿔 볼 수 있을까요?,일단 여행을 다녀 보려고. 그동안 너무 앞만 보고 달려왔어.,여행을 다녀 볼 생각이시군요.
308,많은 고민 후 은퇴를 결심했는데 주변에서 다들 섣부른 생각이라고 해서 마음이 안 좋아.,은퇴를 결심했는데 주변에선 섣부르다고 해서 마음이 안 좋으셨군요.,힘들게 결심했는데 다들 반대하니까 조금 더 고민해 보는 게 좋을까 싶어.,조금 더 고민하는 게 좋을지 생각하시는군요. 어떤 선택을 해야 좋을까요?,한 달 정도 더 고민을 해 보려고. 지금 너무 마음이 지쳤어.,마음이 지쳐 한 달 정도 더 고민해 볼 생각이시군요.
...,...,...,...,...,...,...
51625,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
51627,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [21]:
df2_values = [i[0] for i in df2.values]
human_df1 = pd.DataFrame({'사람문장1':df2_values})

human_df1

,사람문장1
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...
51625,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.
51626,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.
51627,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.


In [24]:
human_df1.values

array([['일은 왜 해도 해도 끝이 없을까? 화가 난다.'],
       ['이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.'],
       ['회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. '],
       ...,
       ['이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.'],
       ['몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.'],
       ['남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.']],
      dtype=object)

In [28]:
import nltk

# Download the 'punkt' resource
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
# Tokenization 함수 정의
def tokenize_sentences(sentences):
    tokenized = [word_tokenize(sentence[0]) for sentence in sentences]
    return tokenized

# Tokenization 수행
tokenized_sentences = tokenize_sentences(human_df1.values)

# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Word2Vec 임베딩 확인 (예시: '화가'라는 단어의 벡터 표현)
embedding_example = word2vec_model.wv['화가']
print("Word2Vec Embedding 예시:")
print(embedding_example)

Word2Vec Embedding 예시:
[-0.10566369 -0.28883874 -1.5681398   0.05160831  0.59623516 -1.8774697
 -0.97151023  2.857401   -0.78301764 -0.81033087  0.3341237  -1.2898769
 -0.6653505   0.3205622  -0.22996123  0.30525512 -0.2385247  -1.2690967
  0.8330063  -2.495355    0.20605032 -0.16781986  0.67748755 -0.98277444
 -0.3403236   0.04501155 -0.56989354 -0.26395133 -1.391114    0.45278174
 -0.13236675 -0.5310943   2.203646   -0.86614907 -0.49238265  0.703234
 -0.71796113 -1.4172379   0.5221058  -1.6532371   0.8224497  -1.7862512
 -0.7510144  -0.5705445   2.0689213  -2.4704266   0.10206217 -0.10447981
 -0.20048538  1.4601712  -1.063227   -0.40890586  0.2502473   0.66527957
  1.0699024   0.7029581  -0.6194679   0.5897006  -1.350234    0.3070983
  0.07614689 -0.31253287  0.01762511  1.1378707  -1.3478835   1.6135283
  1.8516556   2.0197196  -1.9853007   1.1728681  -0.25424916 -0.11874396
  1.8472564  -1.3723079   1.2104402  -0.05542888 -0.05318312  0.7124814
 -1.5370353  -0.39882797  1.0924901  